In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
import wandb

import torch
from torch import nn
from torchvision import models
import torch.optim as optim

from dataset import get_dataloaders, get_tiles_datasets
from utils import seed_everything
from trainer import Trainer

# Params
Image.MAX_IMAGE_PIXELS = 1e11
CFG = {
    'seed': 42,
    'cv_fold': 5,
    'base_model': 'convnext_tiny',   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, efficientnet_v2_s, convnext_tiny, swin_t
    'img_size': 512,
    'batch_size': 32,
    'freeze_epochs': 1,
    'epochs': 5,
    'base_lr': 1e-4,
    'affine_degrees': 0,
    'affine_translate': None,
    'affine_scale': None,
    'dataloader_num_workers': 8,
    'scheduler_step_size': 2,
    'img_color_mean': [0.8708488980328596, 0.75677901508938, 0.8545134911215124],
    'img_color_std': [0.08086288591996027, 0.11553960008706814, 0.06914169213328555],
    'tile_num': 32,
    'optimizer': 'AdamW',
    'scheduler': 'StepLR',
    'loss': 'CrossEntropyLoss',
    'label_smoothing': 0,
    'lr_gamma': 0.1,
    'sgd_momentum': 0.9,
    'valid_type': 'all_tmas',
    'tile_set': 'train_tiles_1024_p50_drop60_v6',
    'tile_table_postfix': '_only_tumor_v2', # '_only_tumor' '_only_tumor_v2'
    'color_jitter': {'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.2},
    'random_erasing_p': 0,
    'fc_2_in_features': 512,
    'fc_dropout': 0.2
}
tags=['torch', 'tiles', 'cv', 'best_epoch']
notes = 'extended double layer head'
plot_samples = False
train_final_model = True

# Wandb
wandb.login()
run = wandb.init(project='kaggle-ubc-ocean', config=CFG, tags=tags, notes=notes)

# Label encoder/decoder
encode = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
decode = {v: k for k, v in encode.items()}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Paths
root = '/media/latlab/MR/projects/kaggle-ubc-ocean'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_image_dir = os.path.join(results_dir, CFG['tile_set'])
train_csv = '{}_sample{}.csv'.format(CFG['tile_set']+CFG['tile_table_postfix'], CFG['tile_num'])

# Seed
seed_everything(CFG['seed'])

# Load data
df = pd.read_csv(os.path.join(results_dir, train_csv))
df['label'] = df.loc[:,'label'].map(encode)

# Functions
def train_model(CFG, train_image_dir, df_train, df_validation, encode, state_filename, validate=True, wandb_log=False):
    # Data loaders
    datasets = get_tiles_datasets(CFG, train_image_dir, df_train, df_validation)
    dataloaders = get_dataloaders(CFG, datasets)

    # Model definition
    model = models.get_model(CFG['base_model'], weights='DEFAULT').to(device)

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    
    # Replace the classifier layer
    if CFG['base_model'].startswith('resnet'):
        model.fc = nn.Linear(in_features=model.fc.in_features, out_features=len(encode)).to(device)
    elif CFG['base_model'].startswith('efficientnet'):
        model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('convnext'):
        model.classifier[2] = nn.Linear(in_features=model.classifier[2].in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('vit'):
        model.heads.head = nn.Linear(in_features=model.heads.head.in_features, out_features=len(encode), bias=True).to(device)
    elif CFG['base_model'].startswith('swin'):
        model.head = nn.Linear(in_features=model.head.in_features, out_features=len(encode), bias=True).to(device)
    else:
        raise NotImplementedError('Classifier layer replacement not implemented for this model')
    
    # Loss function
    if CFG['loss'] == 'CrossEntropyLoss':
        loss_fn = nn.CrossEntropyLoss(label_smoothing=CFG['label_smoothing'])
    elif CFG['loss'] == 'BCEWithLogitsLoss':
        loss_fn = nn.BCEWithLogitsLoss()

    # Optimizer
    if CFG['optimizer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=CFG['base_lr'], momentum=CFG['sgd_momentum'])
    elif CFG['optimizer'] == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=CFG['base_lr'])
    elif CFG['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=CFG['base_lr'])
    
    # Scheduler
    if CFG['scheduler'] == 'StepLR':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=CFG['scheduler_step_size'], gamma=CFG['lr_gamma'], verbose=True)
    elif CFG['scheduler'] == 'CyclicLR':
        scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=CFG['base_lr'], max_lr=CFG['base_lr']*5,
                                                step_size_up=5, cycle_momentum=False, mode='triangular2', verbose=True)
    elif CFG['scheduler'] == 'CosineAnnealingLR':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG['epochs']+CFG['freeze_epochs'], verbose=True)
    elif CFG['scheduler'] == 'OneCycleLR':
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CFG['base_lr'], total_steps=CFG['epochs']+CFG['freeze_epochs'], verbose=True)

    # Training
    trainer = Trainer(model, dataloaders, loss_fn, optimizer, scheduler, device, state_filename=state_filename, metric='balanced_accuracy', wandb_log=wandb_log)
    trainer.train_epochs(num_epochs=CFG['freeze_epochs'], validate=validate)
    trainer.unfreeze()
    trainer.train_epochs(num_epochs=CFG['epochs'], validate=validate)
    trainer.save_state(state_filename)
    return trainer

In [ ]:
# Show training data
if plot_samples:
    dataloaders = get_dataloaders(CFG, get_tiles_datasets(CFG, train_image_dir, df, df))
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloaders['train']):
            plt.figure(figsize=(np.ceil(len(X)/2), 12))
            for i in range(len(X)):
                plt.subplot(int(np.ceil(len(X)/6)), 6, i+1)
                img_data = X[i].permute(1, 2, 0).cpu().numpy()
                # Normalize images for plotting (since there are negative values in tensors)
                img_data_norm = np.clip(((img_data - img_data.mean(axis=(0, 1, 2))) / img_data.std(axis=(0, 1, 2)))/4 + 0.5, 0, 1)
                plt.imshow(img_data_norm)
                plt.title(f'{decode[y[i].item()]}')
            if batch >= 1:
                break

In [ ]:
skf = StratifiedGroupKFold(n_splits=CFG['cv_fold'], random_state=CFG['seed'], shuffle=True)
balanced_acc_list = []
if CFG['valid_type'] == 'all_tmas':
    df_tma = df[df['is_tma']==True].copy().reset_index(drop=True)
    df_wsi = df[df['is_tma']==False].copy().reset_index(drop=True)
    for cv, (train_index, valid_index) in enumerate(skf.split(X=np.zeros(len(df_wsi['label'])), y=df_wsi['label'], groups=df_wsi['orig_image_id'])):
        print(f"Cross-validation fold {cv+1}/{CFG['cv_fold']}")
        df_train = df_wsi.iloc[train_index]
        df_validation = df_wsi.iloc[valid_index]
        df_validation = pd.concat([df_validation, df_tma], ignore_index=True).sample(frac=1).reset_index(drop=True)
        run_name = f'{run.name}-cv{cv+1}'
        state_filename = os.path.join(results_dir, 'models', f'ubc-ocean-{run_name}.pt')
        trainer = train_model(CFG, train_image_dir, df_train, df_validation, encode, state_filename)
        balanced_acc_list.append(trainer.best_metric)
        wandb.log({f'balanced_acc_cv{cv+1}': trainer.best_metric})
elif CFG['valid_type'] == 'standard':
    for cv, (train_index, valid_index) in enumerate(skf.split(X=np.zeros(len(df['label'])), y=df['label'], groups=df['orig_image_id'])):
        print(f"Cross-validation fold {cv+1}/{CFG['cv_fold']}")
        df_train = df.iloc[train_index]
        df_validation = df.iloc[valid_index]
        run_name = f'{run.name}-cv{cv+1}'
        state_filename = os.path.join(results_dir, 'models', f'ubc-ocean-{run_name}.pt')
        trainer = train_model(CFG, train_image_dir, df_train, df_validation, encode, state_filename)
        balanced_acc_list.append(trainer.best_metric)
        wandb.log({f'balanced_acc_cv{cv+1}': trainer.best_metric})
wandb.log({f'mean_balanced_acc': np.mean(balanced_acc_list)})
wandb.finish()

In [ ]:
# Final training on all data
if train_final_model:
    state_filename = os.path.join(results_dir, 'models', f'ubc-ocean-{run.name}.pt')
    trainer = train_model(CFG, train_image_dir, df, df, encode, state_filename, validate=False, wandb_log=False)
    wandb.finish()